# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [RTBF](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [2]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-none"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [4]:
# Affichage du nombre d'articles récupérés
len(articles)

106

In [ ]:
# Affichage des 10 premières entrées
articles[:10]

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/notre-politique-a-...,"""Notre politique a permis d’éviter le bain de ..."
1,https://www.rtbf.be/article/romelu-lukaku-adre...,Romelu Lukaku adresse un message à Eden Hazard...
2,https://www.rtbf.be/article/direct-guerre-isra...,"Direct – Guerre Israël-Gaza : ""Le blocus de Ga..."
3,https://www.rtbf.be/article/guerre-israel-gaza...,Guerre Israël-Gaza : quels termes et quelles i...
4,https://www.rtbf.be/article/budget-federal-lop...,Budget fédéral : l’opposition flingue le bilan...
...,...,...
101,https://www.rtbf.behttps://auvio.rtbf.be/media...,Ici tout commence
102,https://www.rtbf.behttps://auvio.rtbf.be/media...,New Amsterdam S03
103,https://www.rtbf.behttps://auvio.rtbf.be/media...,OPJ Pacifique Sud
104,https://www.rtbf.behttps://auvio.rtbf.be/media...,Tu ne tueras point


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

"Notre politique a permis d’éviter le bain de sang social" : le Premier ministre présente son budget et dresse le bilan de son gouvernement
Le Premier ministre, Alexandre De Croo, a présenté sa déclaration de politique générale, ce mardi après-midi, dès 14h15, à la Chambre. Devant les députés, il a dressé les grandes lignes de la politique que le gouvernement fédéral va mener en 2024, dans sa dernière ligne droite avant les élections législatives de juin prochain.
Ce week-end et ce lundi, la coalition Vivaldi a bouclé ses discussions budgétaires. Parmi les mesures les plus commentées aujourd’hui, figurent la taxation des banques et l’extension du régime des flexi-jobs.
Lors de son exposé, le Premier ministre a aussi dressé le bilan du travail accompli par son gouvernement. " Notre politique a permis de protéger le portefeuille des Belges. Notre politique a permis d’éviter le bain de sang social ", a ainsi déclaré Alexander De Croo, résumant les décisions que la Vivaldi a prises dans le

### Nettoyage du texte à l'aide d'expressions régulières

In [8]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Le Premier ministre, Alexandre De Croo, a présenté sa déclaration de politique générale, ce mardi après-midi, dès 14h15, à la Chambre. Devant les députés, il a dressé les grandes lignes de la politique que le gouvernement fédéral va mener en 2024, dans sa dernière ligne droite avant les élections législatives de juin prochain.
Ce week-end et ce lundi, la coalition Vivaldi a bouclé ses discussions budgétaires. Parmi les mesures les plus commentées aujourd’hui, figurent la taxation des banques et l’extension du régime des flexi-jobs.
Lors de son exposé, le Premier ministre a aussi dressé le bilan du travail accompli par son gouvernement. " Notre politique a permis de protéger le portefeuille des Belges. Notre politique a permis d’éviter le bain de sang social ", a ainsi déclaré Alexander De Croo, résumant les décisions que la Vivaldi a prises dans les contextes de crise du Covid et de crise énergétique.
Avant de présenter sa déclaration de politique générale, le Premier ministre a présen

### Création d'un fichier avec le contenu de l'article


In [ ]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp